In [ ]:
import os
import pandas as pd
from statistics import median

from lib.database.mysql import MySqlConnection
from lib.analyse.workplan import Workplan
from lib.analyse.workday import Workday
from lib.analyse.workweek import WorkWeek



agent = MySqlConnection()

In [ ]:
all_employees_query = "select * from employee order by name asc"

shifts_query = """
select w.date, w.year, w.weeknumber, w.weekday, sc.code, sc.start, sc.end, s.timestamp from shift as s 
	join shiftcode as sc on s.shiftcode_id = sc.id
    join workday as w on s.workday_id = w.id
	where s.employee_id = %s and sc.code != "" and sc.code != "F1" and sc.code != "F3"
    order by w.date asc, s.timestamp desc """

In [ ]:
workplan_list: list[Workplan] = []

employees = agent.query_all(all_employees_query)

for e in employees:
    workplan = Workplan(e[1])
    shifts = agent.query_all(shifts_query, (e[0],))

    last_day = None
    
    for s in shifts:
        this_day = s[0]
        if last_day != this_day: workplan.add_workday(s)
        else: workplan.add_duplicate(s)
        last_day = this_day
    workplan_list.append(workplan)

In [ ]:
worktime_data_list = []
worktime_adj_list = []
above_8_data_list = []
below_8_data_list = []

for item in workplan_list:

    if item.total_workhours == 0: continue

    worktime_result = [item.name, item.workday_count, item.total_workhours, (item.total_workhours / item.workday_count)]
    worktime_data_list.append(worktime_result)

    adjusted_worktime = 0

    wd_keys = item.workdays_dict.keys()
    for k in wd_keys:
        wd:Workday = item.workdays_dict[k]
        time = wd.worktime
        if time < 6: time = 8
        adjusted_worktime += time

    worktime_adj_result = [item.name, item.workday_count, adjusted_worktime, (adjusted_worktime / item.workday_count)]
    worktime_adj_list.append(worktime_adj_result)

    if item.workdays_above_8_count > 0:
        keys = item.workdays_above_8_dict.keys()
        for k in keys:
            wd:Workday = item.workdays_above_8_dict[k]
            data = [item.name, wd.date, wd.code, wd.start, wd.end, wd.worktime]
            above_8_data_list.append(data)
    
    if item.workdays_below_8_count > 0:
        keys = item.workdays_below_8_dict.keys()
        for k in keys:
            wd:Workday = item.workdays_below_8_dict[k]
            if wd.date == "2025-12-24": continue
            if wd.date == "2025-12-31": continue
            data = data = [item.name, wd.date, wd.code, wd.start, wd.end, wd.worktime]
            below_8_data_list.append(data)

In [ ]:
worktime_data_dict = {
    "navn": [x[0] for x in worktime_data_list],
    "dager": [int(x[1]) for x in worktime_data_list],
    "timer": [int(x[2]) for x in worktime_data_list],
    "snitt per dag": [float("{:.3f}".format(x[3])) for x in worktime_data_list],
}

median_day = median(worktime_data_dict["dager"])
median_hours = median(worktime_data_dict["timer"])
median_hour_per_day = median(worktime_data_dict["snitt per dag"])
print(median_day, median_hours, median_hour_per_day)

worktime_data_dict[f"avvik median dag ({median_day})"] = [int(x[1] - median_day) for x in worktime_data_list]
worktime_data_dict[f"avvik median timer ({median_hours})"] = [int(x[2] - median_hours) for x in worktime_data_list]
worktime_data_dict[f"avvik median snitt ({median_hour_per_day})"] = [float("{:.3f}".format(x[3] - median_hour_per_day)) for x in worktime_data_list]

worktime_data_df = pd.DataFrame(worktime_data_dict)
worktime_data_df

In [ ]:
worktime_adj_dict = {
    "navn": [x[0] for x in worktime_adj_list],
    "dager": [int(x[1]) for x in worktime_adj_list],
    "timer": [int(x[2]) for x in worktime_adj_list],
    "snitt per dag": [float("{:.3f}".format(x[3])) for x in worktime_adj_list],
}

median_day = median(worktime_adj_dict["dager"])
median_hours = median(worktime_adj_dict["timer"])
median_hour_per_day = median(worktime_adj_dict["snitt per dag"])
print(median_day, median_hours, median_hour_per_day)

worktime_adj_dict[f"avvik median dag ({median_day})"] = [int(x[1] - median_day) for x in worktime_adj_list]
worktime_adj_dict[f"avvik median timer ({median_hours})"] = [int(x[2] - median_hours) for x in worktime_adj_list]
worktime_adj_dict[f"avvik median snitt ({median_hour_per_day})"] = [float("{:.3f}".format(x[3] - median_hour_per_day)) for x in worktime_adj_list]

worktime_adj_data_df = pd.DataFrame(worktime_adj_dict)
worktime_adj_data_df

In [ ]:
above_8_data_dict = {
    "navn": [x[0] for x in above_8_data_list],
    "dato": [x[1] for x in above_8_data_list],
    "kode": [x[2] for x in above_8_data_list],
    "start": [x[3] for x in above_8_data_list],
    "slutt": [x[4] for x in above_8_data_list],
    "timer": [float("{:.2f}".format(x[5])) for x in above_8_data_list],
}

above_8_df = pd.DataFrame(above_8_data_dict)
above_8_df

In [ ]:
below_8_data_dict = {
    "navn": [x[0] for x in below_8_data_list],
    "dato": [x[1] for x in below_8_data_list],
    "kode": [x[2] for x in below_8_data_list],
    "start": [x[3] for x in below_8_data_list],
    "slutt": [x[4] for x in below_8_data_list],
    "timer": [float("{:.2f}".format(x[5])) for x in below_8_data_list],
}

below_8_df = pd.DataFrame(below_8_data_dict)
below_8_df

In [ ]:
query_shiftcodes = "select * from shiftcode order by start asc, end asc"
shiftcodes_list = agent.query_all(query_shiftcodes)
shiftcodes_dict = {
    "code": [x[1] for x in shiftcodes_list],
    "start": [x[2] for x in shiftcodes_list],
    "end" : [x[3] for x in shiftcodes_list],
}

shiftcode_df = pd.DataFrame(shiftcodes_dict)
shiftcode_df

In [ ]:
path = os.path.join(os.getcwd(), "data", "vaktbok.xlsx")
with pd.ExcelWriter(path) as writer:
    worktime_data_df.to_excel(writer, "arbeidstid statistikk")
    worktime_adj_data_df.to_excel(writer, "arbeidstid, justert")
    above_8_df.to_excel(writer, "over 8 timer")
    below_8_df.to_excel(writer, "under 8 timer")
    shiftcode_df.to_excel(writer, "skiftkoder")
